# 0. Pip Installations

In [45]:
!pip install opendatasets
!pip install polars

     ---------------------------------------- 20.5/20.5 MB 2.3 MB/s eta 0:00:00


# 1. Load dataset from kaggle

In [52]:
import opendatasets as od
import polars as pl

od.download("https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset")
file =('sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv')


df = pl.read_csv(file,separator=',', encoding='ISO-8859-1')

Skipping, found downloaded files in ".\sentiment-analysis-dataset" (use force=True to force download)


In [48]:
df.head()

polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
i64,i64,str,str,str,str
0,1467810672,"""Mon Apr 06 22:…","""NO_QUERY""","""scotthamilton""","""is upset that …"
0,1467810917,"""Mon Apr 06 22:…","""NO_QUERY""","""mattycus""","""@Kenichan I di…"
0,1467811184,"""Mon Apr 06 22:…","""NO_QUERY""","""ElleCTF""","""my whole body …"
0,1467811193,"""Mon Apr 06 22:…","""NO_QUERY""","""Karoli""","""@nationwidecla…"
0,1467811372,"""Mon Apr 06 22:…","""NO_QUERY""","""joy_wolf""","""@Kwesidei not …"


# 2. Data Preprocessing

## 2.1 Data cleaning

In [74]:
df.null_count()
# No null data

polarity,id,date,query,user,text
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


In [54]:
# Rename columns
df.columns = ['polarity', 'id', 'date', 'query', 'user', 'text']

## 2.2 Remove punctuations and special characters 

In [75]:
import string
df = df.with_columns(pl.col('text').map_elements(lambda x: x.translate(str.maketrans('', '', string.punctuation))))
df.head()

polarity,id,date,query,user,text
i64,i64,str,str,str,str
0,1467810672,"""Mon Apr 06 22:…","""NO_QUERY""","""scotthamilton""","""is upset that …"
0,1467810917,"""Mon Apr 06 22:…","""NO_QUERY""","""mattycus""","""Kenichan I div…"
0,1467811184,"""Mon Apr 06 22:…","""NO_QUERY""","""ElleCTF""","""my whole body …"
0,1467811193,"""Mon Apr 06 22:…","""NO_QUERY""","""Karoli""","""nationwideclas…"
0,1467811372,"""Mon Apr 06 22:…","""NO_QUERY""","""joy_wolf""","""Kwesidei not t…"
